## Motivation

In this note we show an example played by hand.  
I hope this helps you understand environments and strategies.   

You will feel tired if you write commands by hand one by one. If so, it's time for you to write your program.  


Reference: 
- Turorial: https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial
- API: https://github.com/Lux-AI-Challenge/Lux-Design-2021/blob/master/kits/README.md
- lux-ai-specifications: https://www.kaggle.com/c/lux-ai-2021/overview/lux-ai-specifications

## Imports
see [tutorial notebook](https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial) for details. 

In [ ]:
!pip install kaggle-environments -U

In [ ]:
# run this if using kaggle notebooks
!cp -r ../input/lux-ai-2021/* .

In [ ]:
from kaggle_environments import make
# pick interesting seed shown in tutorial
# https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial
# env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2}, debug=True)
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 0}, debug=True)

In [ ]:
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys
import random

In [ ]:
DIRECTIONS = Constants.DIRECTIONS

## My Commands
We should return list of commands(strings) such as 'm u_1 w'(unit u_1 move west) or 'r 13 22'(citytile at (13, 22) research).  
It seems hard to make right command from outside, so I use my command and make functions that refers a unit and returns a command.  

My commands fot units

In [ ]:
north = ("move", Constants.DIRECTIONS.NORTH)
east = ("move", Constants.DIRECTIONS.EAST)
south = ("move", Constants.DIRECTIONS.SOUTH)
west = ("move", Constants.DIRECTIONS.WEST)
center = ("move", Constants.DIRECTIONS.CENTER)
city = ("city", )
# transfer = ("transfer", dest_id, resourceType, amount)
pillage = ("pillage", )

My command for citytiles

In [ ]:
research = ("research", )
worker = ("worker", )
cart = ("cart", )

function to return command

In [ ]:
def unit_action(unit, action):
    if action[0] == "move":
        return unit.move(action[1])
    elif action[0] == "city":
        return unit.build_city()
    elif action[0] == "pillage":
        return unit.pillage()
    elif action[0] == "transfer":
        return unit.transfer(action[1], action[2], action[3])  # dest_id, resourceType, amount
    return None


def city_tile_action(city_tile, action):
    if action[0] == "research":
        return city_tile.research()
    elif action[0] == "worker":
        return city_tile.build_worker()
    elif action[0] == "cart":
        return city_tile.build_cart()
    return None

## Hand Agent
refer unit_actions and city_tile_actions writtten in my commands defined below

In [ ]:
game_state = None

def hand_agent(observation, configuration):
    
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    for unit in player.units:
        # print(unit.id)
        if unit.can_act():
            action_tuple = unit_actions[unit.id][observation["step"]]
            action = unit_action(unit, action_tuple)
            if action is not None:
                actions.append(action)   
    for city in player.cities.values():
        for city_tile in city.citytiles:
            # print(city_tile.pos)
            city_tile_key = f"ct_{city_tile.pos.x}_{city_tile.pos.y}"
            if city_tile.can_act():
                action_tuple = city_tile_actions[city_tile_key][observation["step"]]
                action = city_tile_action(city_tile, action_tuple)
                # print(action)
                if action is not None:
                    actions.append(action) 
    # print(actions)
    return actions

actions for citytiles, default is research

In [ ]:
city_tile_actions = {}
for i in range(32):
    for j in range(32):
        city_tile_actions[f"ct_{i}_{j}"] = [research] * 400
city_tile_actions["ct_12_21"][4] = worker
city_tile_actions["ct_12_21"][14] = worker
city_tile_actions["ct_13_22"][20] = worker
city_tile_actions["ct_12_21"][24] = worker
city_tile_actions["ct_12_22"][31] = cart
city_tile_actions["ct_10_21"][32] = cart
city_tile_actions["ct_8_21"][71] = cart

actions for units(workers and carts), default is random move

In [ ]:
u_1_actions = [
    west, north, center, city, west, east, center, west, east, center, 
    west, center, city, west, east, center, south, center, city, west, 
    east, center, west, east, center, west, center, center, city, south, 
    north, center, center, center, center, south, north, center, center, center, 
    
    north, west, north, center, center, center, city, west, east, west, 
    east, west, center, center, city, north, north, east, west, center, 
    city, north, west, west, west, north, center, center, north, center, 
    center, center, center, center, center, center, center, center, center, center, 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]
u_3_actions = [
    center, center, center, center, center, south, east, center, west, east, 
    center, west, east, center, west, east, center, west, north, center, 
    south, east, center, center, center, center, center, center, center, center, 
    south, north, center, center, center, south, north, center, center, center, 
    
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]
u_4_actions = [center] * 10 + [
    center, center, center, center, center, west, west, east, center, west, 
    center, city, west, east, center, west, east, center, west, east,  
    center, south, west, east, center, center, center, center, north, north, 
    
    north, center, west, center, north, center, city, center, south, north, 
    center, west, east, west, east, west, west, city, north, south, 
    north, west, west, north, center, north, center, city, center, center, 
    center, center, center, center, center, center, center, center, 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]
u_5_actions = [center] * 20 + [
    center, south, west, center, north, center, center, center, center, center, 
    city, south, north, center, center, center, west, west, center, north, 
    
    north, center, center, center, center, center, center, west, center, center, 
    south, west, west, north, center, north, center, east, center, east, 
    city, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]
u_6_actions = [center] * 20 + [
    center, center, center, center, center, center, center, center, center, center, 
    center, south, center, center, center, west, west, center, north, west, 
    
    north, center, south, center, city, west, east, west, center, center, 
    city, south, north, south, north, south, south, south, south, south, 
    south, south, south, south, center, west, south, center, center, ("transfer", "u_7", "coal", 20), 
    center, ("transfer", "u_7", "coal", 100), center, center, center, ("transfer", "u_7", "coal", 100), center, center, center, ("transfer", "u_7", "coal", 100), 
    
    center, center, center, center, ("transfer", "u_7", "coal", 100), center, center, center, center, ("transfer", "u_7", "coal", 100), 
    center, center, center, center, ("transfer", "u_9", "coal", 100), center, center, center, center, ("transfer", "u_9", "coal", 100), 
    center, center, center, center, ("transfer", "u_9", "coal", 100), center, center, center, center, ("transfer", "u_9", "coal", 100), 
    center, center, center, center, ("transfer", "u_9", "coal", 100), center, center, center, center, ("transfer", "u_9", "coal", 100), 
    
    center, center, center, center, ("transfer", "u_9", "coal", 100), center, center, center, center, ("transfer", "u_9", "coal", 100), 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]
u_7_actions = [center] * 30 + [
    center, center, west, west, center, center, center, center, center, center, 
    
    west, west, center, center, south, center, south, center, south, center, 
    center, south, center, south, center, south, center, center, south, center, 
    south, center, west, center, center, west, center, east, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, east, north, north, north, north, north, center, 
    north, north, north, north, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    
] + [random.choice([north, east, south, west, center]) for _ in range(400)]  # cart
u_8_actions = [center] * 30 + [
    center, center, center,  center, center, center, center, center, center, center, 
    
    west, west, center, center, north, center, north, center, north, center, 
    center, north, center, west, center, west, center, center, west, center, 
    west, center, north, center, center, north, center, center, east, center, 
    center, center, center, center, center, center, center, center, center, center, 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]  # cart
u_9_actions = [center] * 80 + [
    south, south, south, center, south, center, south, south, center, south, 
    south, center, south, west, south, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
    center, center, center, center, center, center, center, center, center, center, 
] + [random.choice([north, east, south, west, center]) for _ in range(400)]  # cart
unit_actions = {
    'u_1': u_1_actions, 
    'u_3': u_3_actions, 
    'u_4': u_4_actions,
    'u_5': u_5_actions, 
    'u_6': u_6_actions, 
    'u_7': u_7_actions, 
    'u_8': u_8_actions, 
    'u_9': u_9_actions, 
}

## Run and Watch
and edit commans and run and edit commands and run and ...

In [ ]:
env.reset()
_ = env.run([hand_agent, "simple_agent"])
env.render(mode="ipython", width=800, height=600)

# Issues found